In [1]:
from utils import *
from dataset import *
from constants import *
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from dataset import Participant

#### Select participant and session (session only for plot purposes)

In [2]:
participant = Participant('s12', data_path=DATA_PATH_NOTEBOOK)

In [3]:
print(f'Number of sessions: {len(participant.sessions)}')
print(f'Number of channels: {len(participant.channels)}')
print(f'Number of relevant channels: {len(participant.relevant_channels)}')
print(f'The relevant channels are located in the following locations:')
print([participant.channels_locations[i] for i in [channel.idx for channel in participant.relevant_channels]])

Number of sessions: 2
Number of channels: 186
Number of relevant channels: 29
The relevant channels are located in the following locations:
[np.str_('insula'), np.str_('postcentral'), np.str_('postcentral'), np.str_('supramarginal'), np.str_('supramarginal'), np.str_('postcentral'), np.str_('superiorfrontal'), np.str_('insula'), np.str_('insula'), np.str_('insula'), np.str_('WM_insula'), np.str_('posteriorcingulate'), np.str_('posteriorcingulate'), np.str_('posteriorcingulate'), np.str_('posteriorcingulate'), np.str_('WM_paracentral'), np.str_('precentral'), np.str_('precentral'), np.str_('precentral'), np.str_('posteriorcingulate'), np.str_('posteriorcingulate'), np.str_('posteriorcingulate'), np.str_('paracentral'), np.str_('insula'), np.str_('insula'), np.str_('insula'), np.str_('superiorparietal'), np.str_('superiorparietal'), np.str_('superiorparietal')]


#### Get participant's features

In [4]:
features = participant.get_features_all_sessions_ExObs()

100%|██████████| 128/128 [04:26<00:00,  2.09s/it]


Compute PSD for each baselines and activities, then mean it

In [5]:
print(f'The dataset contains {features.shape[0]} samples and {features.shape[1]} features.')

The dataset contains 257 samples and 2089 features.


## Train a model (SVM)

#### Start without any dimensionality reduction

In [6]:
X = features.drop('label', axis=1)
y = features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM
parameters = {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf', 'sigmoid']}
svm = SVC()
clf = GridSearchCV(svm, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)

# Test SVM
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

{'C': 0.1, 'kernel': 'linear'}
Accuracy: 1.00


#### With PCA

In [7]:
X = features.drop('label', axis=1)
y = features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train SVM
parameters = {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf', 'sigmoid']}
svm = SVC()
clf = GridSearchCV(svm, parameters)
clf.fit(X_train_pca, y_train)
print(clf.best_params_)

# Test SVM
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

{'C': 0.1, 'kernel': 'linear'}
Accuracy: 0.98


## Train a model (Random Forest)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Random Forest
n_estimators = [10, 50, 90, 130]
max_depth = [10, 25, 50]
param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}

rf = RandomForestClassifier() 
clf = GridSearchCV(rf, param_grid)
clf.fit(X_train, y_train)
print(clf.best_params_)

# Test Random Forest
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


c:\Users\sarah\miniconda3\envs\ml4scienceTNEproject\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'max_depth': 25, 'n_estimators': 50}
Accuracy: 0.98


## Train model with one frequency band

In [9]:
for band, (high, low) in FREQ_BANDS.items() :
    print('**********************************************************')
    print('**********************************************************')
    print(f'Band: {band}')
    
    freq_band = {band: (high, low)}
    features = participant.get_features_all_sessions_ExObs(freq_band=freq_band)
    
    X = features.drop('label', axis=1)
    y = features['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # SVM
    print('**********************************************************')
    print('SVM')
    
    # Train SVM
    parameters = {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf', 'sigmoid']}
    svm = SVC()
    clf = GridSearchCV(svm, parameters)
    clf.fit(X_train, y_train)
    print(clf.best_params_)

    # Test SVM
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    
    # PCA SVM
    print('**********************************************************')
    print('PCA SVM')
    
    pca = PCA(n_components=100)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Train SVM
    parameters = {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear', 'rbf', 'sigmoid']}
    svm = SVC()
    clf = GridSearchCV(svm, parameters)
    clf.fit(X_train_pca, y_train)
    print(clf.best_params_)

    # Test SVM
    y_pred = clf.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    
    # RANDOM FOREST
    print('**********************************************************')
    print('RANDOM FOREST')
    
    # Train Random Forest
    n_estimators = [10, 50, 90, 130]
    max_depth = [10, 25, 50]
    param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}

    rf = RandomForestClassifier() 
    clf = GridSearchCV(rf, param_grid)
    clf.fit(X_train, y_train)
    print(clf.best_params_)

    # Test Random Forest
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    


**********************************************************
**********************************************************
Band: Delta


100%|██████████| 128/128 [01:30<00:00,  1.41it/s]


**********************************************************
SVM
{'C': 0.1, 'kernel': 'sigmoid'}
Accuracy: 0.87
**********************************************************
PCA SVM
{'C': 0.1, 'kernel': 'sigmoid'}
Accuracy: 0.87
**********************************************************
RANDOM FOREST
{'max_depth': 10, 'n_estimators': 50}
Accuracy: 0.90
**********************************************************
**********************************************************
Band: Theta


100%|██████████| 128/128 [00:53<00:00,  2.38it/s]


**********************************************************
SVM
{'C': 1, 'kernel': 'sigmoid'}
Accuracy: 0.96
**********************************************************
PCA SVM
{'C': 1, 'kernel': 'sigmoid'}
Accuracy: 0.96
**********************************************************
RANDOM FOREST
{'max_depth': 50, 'n_estimators': 10}
Accuracy: 0.98
**********************************************************
**********************************************************
Band: Alpha


...

**********************************************************
SVM
{'C': 10, 'kernel': 'rbf'}
Accuracy: 0.94
**********************************************************
PCA SVM
{'C': 10, 'kernel': 'rbf'}
Accuracy: 0.94
**********************************************************
RANDOM FOREST
{'max_depth': 10, 'n_estimators': 50}
Accuracy: 0.96
**********************************************************
**********************************************************
Band: Gamma


100%|██████████| 128/128 [00:36<00:00,  3.51it/s]


**********************************************************
SVM
{'C': 0.1, 'kernel': 'linear'}
Accuracy: 1.00
**********************************************************
PCA SVM
{'C': 0.1, 'kernel': 'linear'}
Accuracy: 1.00
**********************************************************
RANDOM FOREST
{'max_depth': 10, 'n_estimators': 10}
Accuracy: 1.00
**********************************************************
**********************************************************
Band: HighGamma


100%|██████████| 128/128 [00:36<00:00,  3.53it/s]


**********************************************************
SVM
{'C': 0.1, 'kernel': 'linear'}
Accuracy: 0.98
**********************************************************
PCA SVM
{'C': 10, 'kernel': 'rbf'}
Accuracy: 0.98
**********************************************************
RANDOM FOREST
{'max_depth': 50, 'n_estimators': 130}
Accuracy: 0.98
